# pobranie skryptu scenariusza z `imsdb.com`

In [1]:
import numpy as np, pandas as pd, re

załadowanie stringa ze scriptem

In [2]:
fc = pd.read_html("http://www.imsdb.com/scripts/Fight-Club.html")

tworzymy ramke danych z odpowiedniej tabeli html

In [3]:
skrypt = re.split(r'\s\s', fc[12][0][0])

In [4]:
skryptser = pd.DataFrame(
    {'linia': skrypt})

In [77]:
skryptser.head(3)

linia
scena                                                   
0      <b><!-- </b>if (window!= top) top.location.hre...
0                                      F I G H T C L U B
0                                            by Jim Uhls

In [82]:
#skrypt

### szukam regex

probujemy znalezc wzorzec slow pisanych wielkimi literami

In [6]:
proba = pd.Series(data=['TYLER (V.O)', 'BOB', 'BOB went to home', 'BOB OS'])

In [7]:
proba

0         TYLER (V.O)
1                 BOB
2    BOB went to home
3              BOB OS
dtype: object

In [16]:
patt = re.compile(r'^[A-Z]+(\s\([A-Z.]+\))?$')

In [17]:
proba.apply(func = patt.match)

0    <_sre.SRE_Match object; span=(0, 11), match='T...
1    <_sre.SRE_Match object; span=(0, 3), match='BOB'>
2                                                 None
3                                                 None
dtype: object

### zliczamy per scena per postacie

zakładamy indeks, możemy zliczać per scena

In [18]:
skryptser = skryptser.set_index(skryptser.linia.str.contains(r'\AINT\.')\
            .cumsum())

In [19]:
skryptser.index.name = 'scena'

In [20]:
skryptser.head(2)

linia
scena                                                   
0      <b><!-- </b>if (window!= top) top.location.hre...
0                                      F I G H T C L U B

### filtrujemy zmienna do postaci

indeks ze scenami juz jest, zostawiamy w zmiennej <br>
tylko stringi z imionami postaci

In [21]:
skryptser2 = skryptser[skryptser.linia.apply(func=patt.match).astype(bool)]

In [22]:
skryptser2.index

Int64Index([  0,   1,   1,   1,   1,   1,   1,   1,   1,   1,
            ...
            173, 173, 173, 173, 173, 173, 173, 173, 173, 173],
           dtype='int64', name='scena', length=1208)

In [23]:
skryptser2.head()

linia
scena             
0      JACK (V.O.)
1            TYLER
1             JACK
1      JACK (V.O.)
1             JACK

# FILTRUJEMY DO 3 POSTACI

In [24]:
skryptser2.linia[skryptser2.linia == 'JACK (V.O.)'] = 'JACK'

/home/kamil/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/kamil/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [25]:
skryptser2.linia[~skryptser2.linia.isin(['JACK', 'TYLER', 'MARLA'])] = \
    'OTHER'

/home/kamil/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/kamil/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


### grupujemy, przezywamy indeksy

In [26]:
skrypgrup = skryptser2.groupby(skryptser2.index)['linia'].value_counts()        

In [27]:
skrypgrup.index.set_names(['scena', 'postac'], inplace=True)

In [28]:
final = skrypgrup.reset_index()

In [29]:
final.head()

scena postac  linia
0      0   JACK      1
1      1   JACK      8
2      1  TYLER      3
3      2   JACK      4
4      2  OTHER      3

### sceny z udzialem marli

In [30]:
# sceny z marla
marlasceny = final.scena[final.postac.isin(['MARLA'])].unique()

In [31]:
final = final[final.scena.isin(marlasceny)]

In [32]:
final.loc[final.scena == 28,:]

scena postac  linia
37     28   JACK     21
38     28  MARLA     14
39     28  OTHER      8

# UWAGA: przerobilem postac i `danelong.csv` są dalej

In [67]:
# final.to_csv("danelong.csv")

# Budujemy zmienna dla JACK

In [33]:
jacksceny = final.loc[final.postac == 'JACK', ['scena', 'linia']]
jacksceny_ind = final.scena[final.postac.isin(['JACK'])].unique()

In [34]:
jacksceny.set_index(jacksceny_ind, inplace = True)

In [35]:
jacksceny.head()

scena  linia
16     16      2
28     28     21
29     29      5
30     30      8
69     69     10

In [36]:
marlasceny

array([ 16,  26,  28,  29,  30,  69,  74,  77,  78,  79,  81,  91,  94,
        95,  96, 121, 133, 148, 160, 161, 166, 173])

zbuduj baze, tzn. tabelke SCENA | UDZIAL | INDEX dla Jack

In [37]:
temp_jack = pd.DataFrame({
        "scena": marlasceny
    }, index = marlasceny)

In [38]:
temp_jack["ind"] = pd.Series(temp_jack.scena.isin(jacksceny.scena))

In [39]:
temp_jack.head(10)

scena    ind
16     16   True
26     26  False
28     28   True
29     29   True
30     30   True
69     69   True
74     74   True
77     77  False
78     78  False
79     79  False

przelec petla

In [40]:
for i in temp_jack.index:
    if temp_jack.loc[i,'ind'] == True:
        temp_jack.loc[i,"udzial"] = jacksceny.loc[i,"linia"]
    else:
        temp_jack.loc[i,"udzial"] = 0

In [49]:
temp_jack.head()

scena    ind  udzial
16     16   True     2.0
26     26  False     0.0
28     28   True    21.0
29     29   True     5.0
30     30   True     8.0

# Budujemy zmienna dla TYLER

In [42]:
tylersceny = final.loc[final.postac == 'TYLER', ['scena', 'linia']]
tylersceny_ind = final.scena[final.postac.isin(['TYLER'])].unique()

In [43]:
tylersceny.set_index(tylersceny_ind, inplace = True)

In [44]:
temp_tyler = pd.DataFrame({
        "scena": marlasceny
    }, index = marlasceny)

In [45]:
temp_tyler["ind"] = pd.Series(temp_tyler.scena.isin(tylersceny.scena))

In [46]:
for i in temp_tyler.index:
    if temp_tyler.loc[i,'ind'] == True:
        temp_tyler.loc[i,"udzial"] = tylersceny.loc[i,"linia"]
    else:
        temp_tyler.loc[i,"udzial"] = 0

In [48]:
temp_tyler.head()

scena    ind  udzial
16     16  False     0.0
26     26  False     0.0
28     28  False     0.0
29     29  False     0.0
30     30  False     0.0

# Budujemy zmienna dla OTHER

In [50]:
othersceny = final.loc[final.postac == 'OTHER', ['scena', 'linia']]
othersceny_ind = final.scena[final.postac.isin(['OTHER'])].unique()

othersceny.set_index(othersceny_ind, inplace = True)

temp_other = pd.DataFrame({
        "scena": marlasceny
    }, index = marlasceny)

temp_other["ind"] = pd.Series(temp_other.scena.isin(othersceny.scena))

for i in temp_other.index:
    if temp_other.loc[i,'ind'] == True:
        temp_other.loc[i,"udzial"] = othersceny.loc[i,"linia"]
    else:
        temp_other.loc[i,"udzial"] = 0

In [51]:
temp_other.head()

scena    ind  udzial
16     16  False     0.0
26     26  False     0.0
28     28   True     8.0
29     29  False     0.0
30     30  False     0.0

In [52]:
marlasceny

array([ 16,  26,  28,  29,  30,  69,  74,  77,  78,  79,  81,  91,  94,
        95,  96, 121, 133, 148, 160, 161, 166, 173])

# Zbiorcza tabela

In [53]:
dane = pd.DataFrame({
        #"scena": temp_other.scena,
        "marla": final.loc[final.postac == 'MARLA','linia'].values,
        "jack": temp_jack.udzial.values,
        "tyler": temp_tyler.udzial.values,
        "other": temp_other.udzial.values
    }, index = [x for x in range(len(temp_other))])

In [54]:
dane["suma"] = dane.sum(axis=1)

In [55]:
dane

jack  marla  other  tyler  suma
0    2.0      1    0.0    0.0   3.0
1    0.0      1    0.0    0.0   1.0
2   21.0     14    8.0    0.0  43.0
3    5.0      4    0.0    0.0   9.0
4    8.0      9    0.0    0.0  17.0
5   10.0      9    0.0    0.0  19.0
6    4.0      3    0.0    1.0   8.0
7    0.0      5    0.0    0.0   5.0
8    0.0      3    2.0    0.0   5.0
9    0.0      1    0.0    0.0   1.0
10   0.0      1    0.0    0.0   1.0
11  22.0      6    2.0    9.0  39.0
12  11.0      2    4.0    0.0  17.0
13   3.0      3    0.0    0.0   6.0
14  17.0     10    8.0    0.0  35.0
15  16.0     14    0.0    4.0  34.0
16   7.0      2    0.0    0.0   9.0
17  18.0      9    0.0    8.0  35.0
18  12.0      3    7.0    0.0  22.0
19  21.0     21    2.0    0.0  44.0
20  20.0      1    0.0   27.0  48.0
21  34.0      3    0.0   33.0  70.0

In [56]:
for i in range(4):
    dane.iloc[:,i] = np.round(dane.iloc[:,i]/dane.suma,2)

In [57]:
dane.set_index(marlasceny, inplace = True)

In [61]:
dane = dane.iloc[:,0:4]

In [62]:
dane.head()

jack  marla  other  tyler
16  0.67   0.33   0.00    0.0
26  0.00   1.00   0.00    0.0
28  0.49   0.33   0.19    0.0
29  0.56   0.44   0.00    0.0
30  0.47   0.53   0.00    0.0

In [74]:
dane = dane.reset_index().rename(columns={"index":"scena"})

In [75]:
dane.head()

scena  jack  marla  other  tyler
0     16  0.67   0.33   0.00    0.0
1     26  0.00   1.00   0.00    0.0
2     28  0.49   0.33   0.19    0.0
3     29  0.56   0.44   0.00    0.0
4     30  0.47   0.53   0.00    0.0

In [76]:
dane.to_csv("daneflat.csv", index = False)

# przerabiamy postac danych

In [68]:
dane.head()

jack  marla  other  tyler
16  0.67   0.33   0.00    0.0
26  0.00   1.00   0.00    0.0
28  0.49   0.33   0.19    0.0
29  0.56   0.44   0.00    0.0
30  0.47   0.53   0.00    0.0

In [69]:
dane.stack().reset_index().\
    rename(columns = {"level_0":"scena", "level_1":"postac",0:"udzial"}).to_csv("danelong.csv", index = False)

# WYGENERUJ NAPISY Z DANEJ SCENY

In [90]:
danesceny = skryptser.loc[marlasceny, :]

In [92]:
danesceny.columns

Index(['linia'], dtype='object')

In [ ]:
patt = re.compile(r'^[A-Z]+(\s\([A-Z.]+\))?$')

In [96]:
danesceny['klucz'] = danesceny.linia.apply(func=patt.match).astype(bool).astype(int)

/home/kamil/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [101]:
danesceny = danesceny.reset_index()

In [103]:
danesceny.head()

scena                                              linia  klucz
0     16  INT. FIRST METHODIST CHURCH MEETING ROOM - RES...      0
1     16               Jack's still in an embrace with Bob.      0
2     16                                        JACK (V.O.)      1
3     16                 Bob loved me because he thought my      0
4     16                  testicles were removed too. Being      0

In [108]:
mask = danesceny.linia.str.slice(0,3) == 'INT'

In [115]:
np.where(mask, 1, danesceny.klucz).cumsum()

array([  1,   1,   2, ..., 494, 494, 494])

In [116]:
# danesceny.linia.where(mask, 'NO')
danesceny.klucz = np.where(mask, 1, danesceny.klucz).cumsum()

In [122]:
danesceny.loc[danesceny.scena==26,:]

scena                                              linia  klucz
15     26                     INT. CAVE - JACK'S IMAGINATION      5
16     26  Jack finds Marla smoking a cigarette. Marla co...      5
17     26                     indicating whe wants him to --      5
18     26                                              MARLA      6
19     26                                             Slide.      6

## cum sum

In [118]:
danesceny.to_csv("danesceny.csv", index = False)